## Statistical Data Analysis

In [1]:
from dataloader import get_loader
# root = '/Volumes/Datasets/inner_speech/derivatives/'
root =  'dataset/derivatives/' # -sil
creater = get_loader(root)
xn, yn = creater.load_multiple_subjects([1, 2, 3, 4, 5, 6, 7, 8])

In [2]:
xn.shape

(1240, 128, 1153)

In [3]:
yn.shape

(1240, 4)

## Data Loader strategy

### 1. Load all data of a subject in session 1 and session 2 (dont load 3, because noisy)
### 2. select only inner speech Y[:, 2] ==1
### 3. Stack all subjects recordings along batch axis = 0 (n_recording * n_subjects, 128, 1153)

## Clustering

In [4]:
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [5]:
creater = get_loader(root)
#loader = creater([1, 2])

In [6]:
xn, yn = creater.load_multiple_subjects([1, 2, 3, 4, 5, 6, 7, 8]) 

In [7]:
X = xn.reshape(-1, 128*1153)
print(X.shape)

(1240, 147584)


### PCA transform

In [ ]:
var = 0.98
pca = PCA(var)
pca.fit(X)

In [ ]:
print("Number of components before PCA  = " + str(X.shape[1]))
print("Number of components after PCA 0.98 = " + str(pca.n_components_))

In [ ]:
Clus_dataSet = pca.transform(X)
print("Dimension of our data after PCA = " + str(Clus_dataSet.shape))


### Kmeans clustering

In [ ]:
k_means = KMeans(init = "k-means++", n_clusters = 4, n_init = 10)
k_means.fit(Clus_dataSet)

In [ ]:
k_means_labels = k_means.predict(Clus_dataSet)
k_means_labels

In [ ]:
yn[:, 1]

# Frequency Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Sort data based on classes
labels = ["Up", "Down", "Right", "Left"] 
sorted_data = []
fs = 254
NFFT = 256
classes = yn[:,1]
for i in range(0,4):
    mask = classes == i
    data = xn[mask,:,:]
    sorted_data.append(data)

In [ ]:
# Time Plots
t = np.arange(0,1153) * 1/fs
n = 5 # play with this and figsize to get better images
fig, axs = plt.subplots(nrows=128, ncols=4, figsize=(6*n,150*n))
for i in range(0,4):
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    for j in range(0,128):
        axs[j,i].plot(t, avg_trial[j,:])
        axs[j,i].set_title('Channel: {} Class: {}'.format(j,label))
        axs[j,i].set_xlabel('Time')
        axs[j,i].set_ylabel('Amplitude')
plt.show()

In [ ]:
# Get time plots of channels of interest
chs = np.array([79, 100])
t = np.arange(0,1153) * 1/fs
n = 5 # play with this and figsize to get better images
fig, axs = plt.subplots(nrows=len(chs), ncols=4, figsize=(22,10))
for i in range(0,4):
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    for j, ch in enumerate(chs):
        axs[j,i].plot(t, avg_trial[ch,:])
        axs[j,i].set_title('Channel: {} Class: {}'.format(ch,label))
        axs[j,i].set_xlabel('Time (s)')
        axs[j,i].set_ylabel('Amplitude')
plt.subplots_adjust(hspace=.4)
plt.show()

In [ ]:
# Average and Combined Time Plots
# Average of all channels
t = np.arange(0,1153) * 1/fs
fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(20,15))
for i in range(0,4):
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    if i == 0:
            r = 0
            c = 0
    elif i == 1:
        r = 0
        c = 1
    elif i == 2:
        r = 1
        c = 0
    elif i == 3:
        r = 1
        c = 1
    for j in range(0,128):

        axs[r,c].plot(t, avg_trial[j,:],'b')
        
    avg = np.mean(avg_trial,0)
        
    axs[r,c].plot(t, avg,'r', label='Average')
    axs[r,c].set_title('Class: {}'.format(label))
    axs[r,c].set_xlabel('Time (s)')
    axs[r,c].set_ylabel('Amplitude')
    axs[r,c].legend()


In [ ]:
# Action Interval is between 1-3.5 seconds
fs = 254
start_idx = int(1*fs)
end_idx = int(3.5*fs)


In [ ]:
# Spectrograms
n = 4
fig, axs = plt.subplots(nrows=128, ncols=4, figsize=(4*n,128*n))
for i in range(0,4):
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    #avg_trial = avg_trial[:, start_idx:end_idx]
    for j in range(0,128):
        pxx,  freq, t, cax = axs[j,i].specgram(avg_trial[j,:], Fs=fs, cmap="rainbow", mode='magnitude', NFFT=NFFT, noverlap=NFFT/2)
        axs[j,i].set_title('Channel: {} Class: {}'.format(j,label))
#fig.colorbar(cax)
plt.show()

In [ ]:
# # Get spectrograms of channels of interest
# chs = np.array([79, 100])
# t = np.arange(0,1153) * 1/fs
# n = 5 # play with this and figsize to get better images
# fig, axs = plt.subplots(nrows=len(chs), ncols=4, figsize=(22,10))
# for i in range(0,4):
#     label = labels[i]
#     data = sorted_data[i]
#     avg_trial = np.mean(data, axis=0)
#     for j, ch in enumerate(chs):
#         axs[j,i].plot(t, avg_trial[ch,:])
#         axs[j,i].set_title('Channel: {} Class: {}'.format(ch,label))
#         axs[j,i].set_xlabel('Time (s)')
#         axs[j,i].set_ylabel('Amplitude')
# plt.subplots_adjust(hspace=.4)
# plt.show()

chs = np.array([79, 100])
fig, axs = plt.subplots(nrows=len(chs), ncols=4, figsize=(22,10))
for i in range(0,4):
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    #avg_trial = avg_trial[:, start_idx:end_idx]
    for j, ch in enumerate(chs):
        pxx,  freq, t, cax = axs[j,i].specgram(avg_trial[ch,:], Fs=fs, cmap="rainbow", mode='magnitude', NFFT=NFFT, noverlap=NFFT/2)
        axs[j,i].set_title('Channel: {} Class: {}'.format(j,label))
        axs[j,i].set_xlabel('Time (s)')
        axs[j,i].set_ylabel("Frequency (HZ)")
#fig.colorbar(cax)
plt.subplots_adjust(hspace=.4)
plt.show()

In [ ]:
# Average Spectrograms
# Average of all channels
n = 4
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20,15))
for i in range(0,4):
    if i == 0:
        r = 0
        c = 0
    elif i == 1:
        r = 0
        c = 1
    elif i == 2:
        r = 1
        c = 0
    elif i == 3:
        r = 1
        c = 1
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    avg = np.mean(avg_trial, axis=0)
    pxx,  freq, t, cax = axs[r,c].specgram(avg, Fs=fs, cmap="rainbow", mode='magnitude', NFFT=NFFT, noverlap=NFFT/2)
    axs[r,c].set_title('Class: {}'.format(label))
    axs[r,c].set_xlabel('Time (s)')
    axs[r,c].set_ylabel('Frequency (Hz)')
#fig.colorbar(cax)
plt.show()

In [ ]:
i = 0
label = labels[i]
data = sorted_data[i]
avg_trial = np.mean(data, axis=0)
avg = np.mean(avg_trial, axis=0)


fig, ax = plt.subplots()
pxx,  freq, t, cax = ax.specgram(avg, Fs=fs, cmap="rainbow", mode='magnitude', NFFT=NFFT, noverlap=NFFT/2)
ax.set_title('Class: {}'.format(label))
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
fig.colorbar(cax).set_label('Intensity [dB]')

plt.show()

In [ ]:
# Power Spectral Density Plots
n = 5
fig, axs = plt.subplots(nrows=128, ncols=4, figsize=(6*n,150*n))
for i in range(0,4):
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    avg_trial = avg_trial[:, start_idx:end_idx]
    for j in range(0,128):
        pxx, freqs = axs[j,i].psd(avg_trial[j,:], Fs=fs,NFFT=NFFT, noverlap=NFFT/2, scale_by_freq=False)
        axs[j,i].set_title('Channel: {} Class: {}'.format(j,label))

plt.show()

In [ ]:
# Average PSDs
# Average of all channels
n = 4
fs = 254
start_idx = int(1*fs)
end_idx = int(3.5*fs)
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20,15))
for i in range(0,4):
    if i == 0:
        r = 0
        c = 0
    elif i == 1:
        r = 0
        c = 1
    elif i == 2:
        r = 1
        c = 0
    elif i == 3:
        r = 1
        c = 1
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    avg_trial = avg_trial[:, start_idx:end_idx]
    pows = 1
    pows = np.zeros((128,len(pxx)))
    for j in range(0,128):
        pxx, freqs = axs[r,c].psd(avg_trial[j,:], Fs=fs,NFFT=NFFT, noverlap=NFFT/2, scale_by_freq=False, c='b')
        pows[j,:] = pxx
    
    avg = np.mean(avg_trial, axis=0)
    #avg = np.mean(pows, axis=0)
    #axs[i].plot(freqs, avg, 'r', label="Average")
    axs[r,c].psd(avg, Fs=fs,NFFT=NFFT, noverlap=NFFT/4, scale_by_freq=False, c='r', label="Average")
    axs[r,c].set_title('Class: {}'.format(label))
    axs[r,c].legend()
    

plt.show()

In [ ]:
# Power Spectral Density Plots
n = 5
fig, axs = plt.subplots(nrows=128, ncols=4, figsize=(6*n,150*n))
for i in range(0,4):
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    avg_trial = avg_trial[:, start_idx:end_idx]
    for j in range(0,128):
        pxx, freqs = axs[j,i].psd(avg_trial[j,:], Fs=fs,NFFT=NFFT, noverlap=NFFT/2, scale_by_freq=False)
        axs[j,i].set_title('Channel: {} Class: {}'.format(j,label))

plt.show()

In [ ]:
# Average PSDs
# Average of all channels
n = 4
fs = 254
start_idx = int(1*fs)
end_idx = int(3.5*fs)
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20,15))
for i in range(0,4):
    if i == 0:
        r = 0
        c = 0
    elif i == 1:
        r = 0
        c = 1
    elif i == 2:
        r = 1
        c = 0
    elif i == 3:
        r = 1
        c = 1
    label = labels[i]
    data = sorted_data[i]
    avg_trial = np.mean(data, axis=0)
    avg_trial = avg_trial[:, start_idx:end_idx]
    pows = 1
    pows = np.zeros((128,len(pxx)))
    for j in range(0,128):
        pxx, freqs = axs[r,c].psd(avg_trial[j,:], Fs=fs,NFFT=NFFT, noverlap=NFFT/2, scale_by_freq=False, c='b')
        pows[j,:] = pxx
    
    avg = np.mean(avg_trial, axis=0)
    #avg = np.mean(pows, axis=0)
    #axs[i].plot(freqs, avg, 'r', label="Average")
    axs[r,c].psd(avg, Fs=fs,NFFT=NFFT, noverlap=NFFT/4, scale_by_freq=False, c='r', label="Average")
    axs[r,c].set_title('Class: {}'.format(label))
    axs[r,c].legend()
    

plt.show()

